In [8]:
from autopy import *
import scanner

In [9]:
# Cache screen size
screen_width, screen_height= screen.get_size()

Scanner = scanner.Scanner();

# COLOR DEFINITIONS
# This is the Dino's colour, also used by Obstacles.
COLOR_DINOSAUR = '535353';

In [14]:
class GameManipulator:
    def __init__(self):
        # Stores the game position (Globally)
        self.offset= None
        self.width= None

        # Stores points (jumps)
        self.points= 0

        # Listners
        self.onGameEnd= None
        self.onGameStart= None
        self.onSensorData= None

        # Game State
        self.gamestate= 'OVER'

        # GameOver Position
        self.gameOverOffset= [190, -82]

        # Stores an array of "sensors" (Ray tracings)
        # Positions are always relative to global "offset"
        self.sensors= [
            {
              'lastValue': 1,

              'value': None,
              'offset': [84, -15], # 64,-15
              'step': [4, 0],
              'length': 0.3,

              # Speed
              'speed': 0,
              'lastComputeSpeed': 0,

              # Computes size of the object
              'size': 0,
              'computeSize': True,
            },
          ]

    # Find out dinosaur (fast)
    def findGamePosition(self):
       
        skipXFast = 15;

        for x in range(20, screen_width,skipXFast):
            dinoPos = Scanner.scanUntil(
                                          # Start position
                                          [x, 80],
                                          # Skip pixels
                                          [0, skipXFast],
                                          # Searching Color
                                          COLOR_DINOSAUR,
                                          # Normal mode (not inverse)
                                          False,
                                          # Iteration limit
                                          500 / skipXFast)

            if (dinoPos):break
        if (dinoPos==False):return None
        
        for x in range(dinoPos[0] - 50, dinoPos[0]+1):
            pos = Scanner.scanUntil(
                                      # Start position
                                      [x, dinoPos[1] - 2],
                                      # Skip pixels
                                      [0, 1],
                                      # Searching Color
                                      COLOR_DINOSAUR,
                                      # Normal mode (not inverse)
                                      False,
                                      # Iteration limit
                                      100)

            if (pos): break


        # Did actually found? If not, error!
        if (pos==False):   return None
  

        # Find the end of the game
        endPos = pos;

        while (bitmap.capture_screen().get_color(endPos[0] + 3, endPos[1]) == COLOR_DINOSAUR):
            endPos = Scanner.scanUntil(
                                        # Start position
                                        [endPos[0] + 2, endPos[1]],
                                        # Skip pixels
                                        [2, 0],
                                        # Searching Color
                                        COLOR_DINOSAUR,
                                        # Invert mode
                                        True,
                                        # Iteration limit
                                        600);
        # Did actually found? If not, error!
        if (endPos==False): return None
  
        #Save to allow global access
        self.offset = pos;
        self.width = 600;#endPos[0] - pos[0];

        return pos

In [15]:
g=GameManipulator()
g.findGamePosition()

KeyboardInterrupt: 